In [11]:
import os
import random
import shutil
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [13]:
real_videos_path = "C:\\Users\\HP\\Downloads\\Celeb-real"
fake_videos_path = "C:\\Users\\HP\\Downloads\\Celeb-synthesis"
output_real_path = 'C:\\Users\\HP\\Downloads\\extracted_real_frames1'
output_fake_path = 'C:\\Users\\HP\\Downloads\\extracted_fake_frames1'

In [15]:
def extract_frames(video_path, save_dir, frame_rate=30):
    """Extract frames from a video file and save them as images."""
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    cap = cv2.VideoCapture(video_path)
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count % frame_rate == 0:  # Save one frame per second
            frame_filename = os.path.join(save_dir, f"frame_{count}.jpg")
            cv2.imwrite(frame_filename, frame)
        count += 1
    cap.release()

In [17]:
def load_images_from_folder(folder, label, img_size=(224, 224)):
    data = []
    labels = []
    for subfolder in os.listdir(folder):  # Each subfolder contains frames for a single video
        subfolder_path = os.path.join(folder, subfolder)
        for filename in os.listdir(subfolder_path):
            img_path = os.path.join(subfolder_path, filename)
            img = load_img(img_path, target_size=img_size)
            img = img_to_array(img) / 255.0  # Normalize images
            data.append(img)
            labels.append(label)
    return np.array(data), np.array(labels)

In [19]:
real_images, real_labels = load_images_from_folder(output_real_path, label=0)
fake_images, fake_labels = load_images_from_folder(output_fake_path, label=1)


In [21]:
X = np.concatenate([real_images, fake_images], axis=0)
y = np.concatenate([real_labels, fake_labels], axis=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [25]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 28s 0us/step


In [27]:
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])


In [31]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

In [33]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
'''Rotation, shifts, and flips introduce variations to the images, helping the model generalize better by training it on multiple perspectives of a face.
This helps the model identify deepfake manipulations even when the face is rotated, slightly shifted, or zoomed.'''

In [35]:
train_generator = datagen.flow(X_train, y_train, batch_size=32)

In [37]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

In [41]:
history = model.fit(train_generator, epochs=10, validation_data=(X_test, y_test),
                    callbacks=[early_stopping, lr_reduce])

Epoch 1/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 1516s 5s/step - accuracy: 0.6990 - loss: 0.6070 - val_accuracy: 0.6685 - val_loss: 0.6387 - learning_rate: 1.0000e-04
Epoch 2/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 1470s 5s/step - accuracy: 0.8609 - loss: 0.3042 - val_accuracy: 0.5648 - val_loss: 0.8418 - learning_rate: 1.0000e-04
Epoch 3/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 1463s 5s/step - accuracy: 0.9033 - loss: 0.2206 - val_accuracy: 0.7062 - val_loss: 0.7930 - learning_rate: 1.0000e-04
Epoch 4/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 1493s 5s/step - accuracy: 0.9232 - loss: 0.1892 - val_accuracy: 0.8920 - val_loss: 0.2396 - learning_rate: 1.0000e-04
Epoch 5/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 1476s 5s/step - accuracy: 0.9368 - loss: 0.1544 - val_accuracy: 0.9208 - val_loss: 0.4266 - learning_rate: 1.0000e-04
Epoch 6/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 1437s 5s/step - accuracy: 0.9380 - loss: 0.1507 - val_accuracy: 0.9183 - val_loss: 0.1878 - learning_rate: 1.0000e-04
Epoch 7/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 1435s 5s/ste

In [43]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

74/74 ━━━━━━━━━━━━━━━━━━━━ 76s 1s/step - accuracy: 0.9708 - loss: 0.0789
Test Accuracy: 96.87%


In [47]:
model.save('C:\\Users\\HP\\Downloads\\deepfake_resnet50_model.keras')